In [1]:
import pandas as pd
import os
import psycopg2
import csv
from scipy import stats

In [2]:
file_names = []
for (dirpath, dirnames, filenames) in os.walk('data'):
    file_names.extend(filenames)
    break
    
file_names = list(filter(lambda x:'csv' in x, filenames))
file_names

['subjects.csv',
 'subject_memberships.csv',
 'rooms.csv',
 'schedules.csv',
 'sections.csv',
 'courses.csv',
 'course_offerings.csv',
 'instructors.csv',
 'teachings.csv',
 'grade_distributions.csv']

In [5]:
tables = {}
for table in file_names:
    with open('data/' + table) as f:
        tables[table[:-4]] = pd.read_csv(f)
        
subjects = tables['subjects']
subject_memberships = tables['subject_memberships']
rooms = tables['rooms']
schedules = tables['schedules']
sections = tables['sections']
courses = tables['courses']
course_offerings = tables['course_offerings']
instructors = tables['instructors']
teachings = tables['teachings']
grade_distributions = tables['grade_distributions']
        
print(f'converted into {len(tables)} dataframes stored in dict: tables')

converted into 10 dataframes stored in dict: tables


In [4]:
# table_cols = {}
# for table in tables.keys():
#     table_cols[table] = list(tables[table].columns)
# for k, v in table_cols.items():
#     print(f'{k} : {v}')

subjects : ['code', 'name', 'abbreviation']
subject_memberships : ['subject_code', 'course_offering_uuid']
rooms : ['uuid', 'facility_code', 'room_code']
schedules : ['uuid', 'start_time', 'end_time', 'mon', 'tues', 'wed', 'thurs', 'fri', 'sat', 'sun']
sections : ['uuid', 'course_offering_uuid', 'section_type', 'number', 'room_uuid', 'schedule_uuid']
courses : ['uuid', 'name', 'number']
course_offerings : ['uuid', 'course_uuid', 'term_code', 'name']
instructors : ['id', 'name']
teachings : ['instructor_id', 'section_uuid']
grade_distributions : ['course_offering_uuid', 'section_number', 'a_count', 'ab_count', 'b_count', 'bc_count', 'c_count', 'd_count', 'f_count', 's_count', 'u_count', 'cr_count', 'n_count', 'p_count', 'i_count', 'nw_count', 'nr_count', 'other_count']


In [6]:
# conn = psycopg2.connect("dbname=uw_mad_grades")
# cur = conn.cursor()

# # for table in file_names:
# #     with open(table, 'r') as f:
# #         num_cols = len(tables[table[:-4]].columns)
# #         cols = "%s," * num_cols
# #         reader = csv.reader(f)
# #         next(reader)  # Skip the header row.
# #         for row in reader:
# #             query = "INSERT INTO " + table[:-4] + " VALUES (" + cols[:-1] + ")"
# #             cur.execute(query, row)
            
# #     conn.commit()
# query = '''
# SELECT
#    sections.course_offering_uuid ,instructor_id, a_count, count(grade_distributions.course_offering_uuid)
# FROM
#    teachings
#    INNER JOIN
#    sections
#    ON teachings.section_uuid=sections.uuid
#    INNER JOIN
#    grade_distributions
#    ON sections.course_offering_uuid=grade_distributions.course_offering_uuid
#    WHERE
#        (CAST(grade_distributions.a_count AS integer) > 0) OR
#        (CAST(grade_distributions.ab_count AS integer) > 0) OR
#        (CAST(grade_distributions.b_count AS integer) > 0) OR
#        (CAST(grade_distributions.bc_count AS integer) > 0) OR
#        (CAST(grade_distributions.c_count AS integer) > 0) OR
#        (CAST(grade_distributions.d_count AS integer) > 0) OR
#        (CAST(grade_distributions.f_count AS integer) > 0)
#    GROUP BY sections.course_offering_uuid, instructor_id, a_count
#    HAVING count(grade_distributions.course_offering_uuid) > 1;'''

# cur.execute(query)
# test_df = pd.DataFrame(cur.fetchall())
# test_df.columns = [i[0] for i in cur.description]

In [7]:
conn = psycopg2.connect("dbname=uw_mad_grades")
cur = conn.cursor()
query = '''
DROP TABLE IF EXISTS grade_dist_numeric
'''
cur.execute(query)
query = '''
CREATE TABLE grade_dist_numeric AS (
  SELECT
    course_offering_uuid
  , section_number
  , CAST(grade_distributions.a_count AS integer)
  , CAST(grade_distributions.ab_count AS integer)
  , CAST(grade_distributions.b_count AS integer)
  , CAST(grade_distributions.bc_count AS integer)
  , CAST(grade_distributions.c_count AS integer)
  , CAST(grade_distributions.d_count AS integer)
  , CAST(grade_distributions.f_count AS integer)
  FROM grade_distributions
  )
'''
cur.execute(query)
conn.commit()

In [8]:
conn = psycopg2.connect("dbname=uw_mad_grades")
cur = conn.cursor()

query = '''
SELECT
  sections.course_offering_uuid
, course_offerings.course_uuid
, course_offerings.name
, COUNT(sections.number) AS section_count
, instructor_id
, SUM(a_count) AS a_count
, SUM(ab_count) AS ab_count
, SUM(b_count) AS b_count
, SUM(bc_count) AS bc_count
, SUM(c_count) AS c_count
, SUM(d_count) AS d_count
, SUM(f_count) AS f_count
FROM
   teachings
INNER JOIN
   sections
   ON teachings.section_uuid=sections.uuid
INNER JOIN
   grade_dist_numeric
   ON (
     (sections.course_offering_uuid = grade_dist_numeric.course_offering_uuid)
      AND (sections.number = grade_dist_numeric.section_number))
INNER JOIN
   course_offerings
   ON sections.course_offering_uuid=course_offerings.uuid
WHERE
   (a_count + ab_count + b_count + bc_count + c_count + d_count + f_count) > 0
GROUP BY
  sections.course_offering_uuid
, course_offerings.course_uuid
, course_offerings.name
, instructor_id
HAVING
  count(instructor_id) > 1;
'''

cur.execute(query)
all_grades_count_df = pd.DataFrame(cur.fetchall())
all_grades_count_df.columns = [i[0] for i in cur.description]

In [9]:
all_grades_count_df.head()

,course_offering_uuid,course_uuid,name,section_count,instructor_id,a_count,ab_count,b_count,bc_count,c_count,d_count,f_count
0,000d1ef4-b78a-304e-8ff9-382dfc96835a,10362c9c-bbf8-39e3-8506-c2783ee31625,Diversity in Special Education,3,3802457,96,15,0,0,0,0,0
1,00193853-8f6d-3e14-994b-77a557b35d74,a4dcc153-1a04-39d8-9b28-b56fb4fa5266,Cult Anthro&Human Diversity,2,3409966,368,272,109,42,21,8,6
2,0019cbe1-e2dc-3c25-a625-bbede76db115,366714bd-5513-3f00-8af3-1f250ecb4062,Plant Physiology,2,2600006,14,6,11,5,4,1,0
3,001d3a50-91d2-3f63-8858-778dc542bbbc,9be02b62-4325-3877-9271-09764cd00bb2,Scand Lit in Tran-20th C,2,2602070,23,36,17,13,9,0,0
4,002d324d-b562-3772-8369-52307754894f,20b909fd-f617-3a90-8871-f9d8efe57dc8,The Teaching of Children's Lit,2,103165,44,1,0,0,0,0,0


In [10]:
col = ['a_count', 'ab_count', 'b_count', 'bc_count', 'c_count', 'd_count', 'f_count']
all_grades_count_df['class_size'] = all_grades_count_df[col].sum(axis=1)
all_grades_count_df['counts'] = all_grades_count_df.groupby(['course_uuid'])['course_uuid'].transform('count')

In [12]:
#Which class was taught the most times?
all_grades_count_df = all_grades_count_df.loc[all_grades_count_df['counts']>1]
all_grades_count_df.sort_values('counts', ascending=False).head()

,course_offering_uuid,course_uuid,name,section_count,instructor_id,a_count,ab_count,b_count,bc_count,c_count,d_count,f_count,class_size,counts
5794,78a11bde-275c-313a-9c65-ea93faacdeba,7a552326-da08-354c-aafc-f56edcf83f44,Intro to Speech Composition,2,5516801,5,14,2,0,0,0,1,22,489
8367,b2695db3-d8eb-31c0-b4f3-add51eb32d03,7a552326-da08-354c-aafc-f56edcf83f44,Intro to Speech Composition,2,3337031,7,7,7,1,1,0,0,23,489
6027,7f44df10-881a-38b9-bb4f-70e00d3444b2,7a552326-da08-354c-aafc-f56edcf83f44,Intro to Speech Composition,2,4709223,5,12,8,0,1,0,0,26,489
599,0b9a80f6-833a-3a56-9a67-7f40652405de,7a552326-da08-354c-aafc-f56edcf83f44,Intro to Speech Composition,2,1441959,4,14,6,0,1,0,0,25,489
600,0b9a80f6-833a-3a56-9a67-7f40652405de,7a552326-da08-354c-aafc-f56edcf83f44,Intro to Speech Composition,2,1544049,4,5,2,2,1,3,0,17,489


In [14]:
prof_matters = all_grades_count_df.loc[all_grades_count_df['course_uuid'] == '7a552326-da08-354c-aafc-f56edcf83f44'].drop(['course_offering_uuid', 'course_uuid', 'section_count', 'name', 'counts'], axis=1)
prof_matters.head()

In [24]:
grade = 'ab'
x = grade.lower()+'_count'
x

'ab_count'

In [49]:
def prof_a_size(df, grade='a'):
    '''Input a dataframe with grade counts and the professors who gave those grades, along with the 
       exact grade letter of interest. Output two dictionaries, each with professor as the keys, 
       ( {prof : df} , {prof : [%<grade> of THIS prof, %<grade> of OTHER profs]})'''
    profs = df['instructor_id'].values
    profs_dict = {}
    profs_pct = {}
    grade_search = grade.lower() + '_count'
    for prof in profs:
        other_profs = df.loc[df['instructor_id'] != prof, ['a_count', 'class_size']]
        pct_other = other_profs.sum()[0] / other_profs.sum()[1]
        profs_dict[prof] = df.loc[df['instructor_id'] == prof, ['a_count', 'class_size']]   #str(grade_search)
        pct_a = profs_dict[prof].sum()[0] / profs_dict[prof].sum()[1]
        profs_pct[prof] = pct_a, pct_other
        
    return profs_dict, profs_pct

profs_dict, profs_pct = prof_a_size(prof_matters, 'a')


In [42]:
[(key, val) for (key, val) in sorted(profs_pct.items(), key=lambda x: x[1][0], reverse=True)][:10]

[('3960315', (0.8, 0.23397720843772732)),
 ('4402193', (0.7551020408163265, 0.23305530812211514)),
 ('4619996', (0.7345132743362832, 0.23052503052503054)),
 ('4398942', (0.6923076923076923, 0.2341577756223731)),
 ('5905783', (0.6666666666666666, 0.2342815581057055)),
 ('5703925', (0.6538461538461539, 0.23423860329776916)),
 ('4070207', (0.6538461538461539, 0.23423860329776916)),
 ('5905700', (0.64, 0.23430049300897116)),
 ('4739455', (0.6212121212121212, 0.22885245901639345)),
 ('3332416', (0.6190476190476191, 0.2344671568231397))]

In [50]:
profs_dict

{'1441959':       a_count  class_size
 599         4          25
 2589        6          25
 3685       12          23
 4762        5          25
 7575        4          26, '1544049':       a_count  class_size
 600         4          17
 2590        2          23
 3686        2          21
 7576        5          21, '3010500':       a_count  class_size
 601         2          22
 3687        0          26, '3196989':       a_count  class_size
 602         6          21
 3690        7          22, '3292045':      a_count  class_size
 603        5          23, '3338326':       a_count  class_size
 604         4          22
 2596        2          25
 3691        1          23, '3404244':       a_count  class_size
 605         3          20
 2597        0          23
 3692        3          25, '359342':        a_count  class_size
 606          5          20
 3539         3          23
 3694         2          24
 6016         5          25
 8369         5          24
 8953         1   

In [27]:
def extreme_profs(n=3, profs_dict=profs_dict, profs_pct=profs_pct):
    i=0
    diff = []
    
    profs_pct_sorted = [(key, val) for (key, val) in sorted(profs_pct.items(), key=lambda x: x[1][0], reverse=True)]
    for prof, pct_tuple in profs_pct_sorted:
        print(f"{prof} gives {round(pct[0] * 100, 1)}% {, while all other profs give {round(pct[1] * 100, 1)}%")
        diff.append([prof, pct[0] - pct[1]])
    
    for prof, df in profs_dict.items():
        display(df)
        i += 1
        if i>n:
            break

,f_count,class_size
599,0,25
2589,1,25
3685,0,23
4762,0,25
7575,0,26


,f_count,class_size
600,0,17
2590,1,23
3686,0,21
7576,0,21


,f_count,class_size
601,0,22
3687,0,26


,f_count,class_size
602,0,21
3690,0,22


,f_count,class_size
603,0,23


,f_count,class_size
604,0,22
2596,0,25
3691,0,23


1441959 gives 0.8% F's, while all other profs give 0.6%
1544049 gives 1.2% F's, while all other profs give 0.6%
3010500 gives 0.0% F's, while all other profs give 0.6%
3196989 gives 0.0% F's, while all other profs give 0.6%
3292045 gives 0.0% F's, while all other profs give 0.6%
3338326 gives 0.0% F's, while all other profs give 0.6%
3404244 gives 2.9% F's, while all other profs give 0.6%
359342 gives 0.6% F's, while all other profs give 0.6%
3614745 gives 0.2% F's, while all other profs give 0.6%
3617728 gives 4.5% F's, while all other profs give 0.6%
3780094 gives 0.0% F's, while all other profs give 0.6%
3784713 gives 0.0% F's, while all other profs give 0.6%
3978152 gives 0.0% F's, while all other profs give 0.6%
3985597 gives 0.0% F's, while all other profs give 0.6%
4124009 gives 0.0% F's, while all other profs give 0.6%
4150938 gives 2.1% F's, while all other profs give 0.6%
912304 gives 2.2% F's, while all other profs give 0.6%
195426 gives 0.0% F's, while all other profs give 

[['1441959', 0.0022799308267673077],
 ['1544049', 0.006430263230855758],
 ['3010500', -0.005829959514170041],
 ['3196989', -0.0058276001618777825],
 ['3292045', -0.005818181818181818],
 ['3338326', -0.005840363400389357],
 ['3404244', 0.023734554649110253],
 ['359342', 0.00037028965327973694],
 ['3614745', -0.003489993315954409],
 ['3617728', 0.03985934597352048],
 ['3780094', -0.005850804485616773],
 ['3784713', -0.005828543673601554],
 ['3978152', -0.005851755526657998],
 ['3985597', -0.005828071879553181],
 ['4124009', -0.005849378503534],
 ['4150938', 0.01508434547908232],
 ['912304', 0.015991073439963956],
 ['195426', -0.005840363400389357],
 ['3077099', 0.002621131521262304],
 ['3473033', -0.0058276001618777825],
 ['3625028', -0.005838942502635633],
 ['3801942', 0.0027657203782926456],
 ['4138221', -0.005828543673601554],
 ['4265924', 0.01536296239257006],
 ['4275432', -0.005935696619950536],
 ['4286663', -0.005849378503534],
 ['4289299', -0.00587515299877601],
 ['4289376', -0.00

In [60]:
display(teachings.head(2))
sections.head(2)

,instructor_id,section_uuid
0,761703,45adf63c-48c9-3659-8561-07556d2d4ddf
1,761703,c6280e23-5e43-3859-893e-540d94993529


,uuid,course_offering_uuid,section_type,number,room_uuid,schedule_uuid
0,45adf63c-48c9-3659-8561-07556d2d4ddf,344b3ebe-da7e-314c-83ed-9425269695fd,FLD,1,NaN,f2d66a4d-0c08-3b48-abf6-649fffd7ae90
1,c6280e23-5e43-3859-893e-540d94993529,f718e6cd-33f0-3c14-a9a6-834d9c3610a8,FLD,1,NaN,f2d66a4d-0c08-3b48-abf6-649fffd7ae90


In [137]:
A = pd.merge(teachings, sections, left_on='section_uuid', right_on='uuid').drop('uuid', axis=1)

In [138]:
A.head()

,instructor_id,section_uuid,course_offering_uuid,section_type,number,room_uuid,schedule_uuid
0,761703,45adf63c-48c9-3659-8561-07556d2d4ddf,344b3ebe-da7e-314c-83ed-9425269695fd,FLD,1,NaN,f2d66a4d-0c08-3b48-abf6-649fffd7ae90
1,761703,c6280e23-5e43-3859-893e-540d94993529,f718e6cd-33f0-3c14-a9a6-834d9c3610a8,FLD,1,NaN,f2d66a4d-0c08-3b48-abf6-649fffd7ae90
2,761703,9395dc21-15d1-3fab-8d1f-6f3fe6114c48,ea3b717c-d66b-30dc-8b37-964d9688295f,FLD,1,04368a56-c959-3e4b-8b3d-f4cc3538fea5,f2d66a4d-0c08-3b48-abf6-649fffd7ae90
3,3677061,b99e440b-39db-350a-81eb-b6eb1bd8b0bc,ea3b717c-d66b-30dc-8b37-964d9688295f,FLD,1,NaN,f2d66a4d-0c08-3b48-abf6-649fffd7ae90
4,761703,ca1c841f-41d5-3298-bd25-01c65751f150,ea3b717c-d66b-30dc-8b37-964d9688295f,FLD,1,NaN,f2d66a4d-0c08-3b48-abf6-649fffd7ae90


In [139]:
B = pd.merge(A, grade_distributions)

In [140]:
cols = B.columns
cols_to_keep = cols.drop(['section_uuid', 'section_type', 'number', 'room_uuid', 'schedule_uuid', 'section_number'])
cols_to_keep
C = B.loc[:, cols_to_keep].groupby('course_offering_uuid')

In [141]:
D = C.head(len(C))

In [142]:
D.drop_duplicates(inplace=True)

In [143]:
D = D.loc[:,['instructor_id', 'course_offering_uuid', 'a_count', 'ab_count',
       'b_count', 'bc_count', 'c_count', 'd_count', 'f_count']]

In [159]:
D['course_offering_uuid'].value_counts()

a2c8781f-b2a3-33ea-b606-740ae7d6ef3f    2208
ceccaea8-0e3a-3828-9572-4db04fb73f28    2116
d6a6d456-7a35-3542-af1d-c8af71328005    2024
12015c86-bd83-3d99-93b6-d12cd2c0f5d6    1980
7ef6ad64-6ab5-3902-8521-253d6428e49d    1978
ad76f040-a3b8-3432-aa1c-6ee66c8550a1    1932
35a65d4a-702c-388e-b98b-a0127e587f16    1800
bdc6679e-2bfd-36a6-8e0a-7639e47ca25a    1770
0f80ce84-1e1f-37e9-b93a-0d2542cb6380    1764
525970c4-dbf3-30ff-8191-f1e7daded574    1740
5fe1b603-04cf-33ea-a7b9-04e6e357c4a2    1710
b2695db3-d8eb-31c0-b4f3-add51eb32d03    1710
41e6b821-0d66-3ecc-925c-ffb16de7ab1d    1682
1fcf43ea-18d8-35a0-9822-5789c0a83920    1681
6b76c9d2-33d9-3c7c-bc33-2b04cd3b6a46    1672
6332a3af-7275-3786-b69d-b754be37b9f2    1650
21ced15c-d665-38b3-abc6-23e523733326    1624
80cd3cee-98d2-3b2d-8cbb-b0bc2f44497d    1599
4c231a8a-2dfc-3994-9a06-afa431164181    1596
cd11d10d-1e78-3d36-ac3a-4a8f1e7ea68c    1590
bbe66764-b560-3fe2-b35a-195eb60e6112    1568
78a11bde-275c-313a-9c65-ea93faacdeba    1568
39a77885-e

In [160]:
D[col] = D[col].apply(pd.to_numeric, errors='coerce')
D['class_size'] = D[['a_count', 'ab_count', 'b_count', 'bc_count', 'c_count', 'd_count', 'f_count']].sum(axis=1)
D['counts'] = D.groupby(['course_offering_uuid'])['course_offering_uuid'].transform('count')

In [161]:
E = D.loc[D['course_offering_uuid'] == 'a2c8781f-b2a3-33ea-b606-740ae7d6ef3f']

In [162]:
E['instructor_id'].value_counts()

942078     46
3467209    46
4284613    46
403594     46
3778398    46
3628821    46
4148887    46
3464767    46
4269584    46
4150782    46
3442158    46
4140447    46
296194     46
773349     46
4140245    46
3454094    46
3976301    46
4150232    46
3117704    46
3981948    46
3457652    46
3123795    46
4080122    46
215332     46
3772829    46
4147838    46
3958227    46
4149156    46
3960722    46
4149090    46
1544018    46
3467035    46
3024437    46
542249     46
3466787    46
4150791    46
298410     46
3973771    46
3982643    46
3630373    46
3343365    46
4146113    46
4150119    46
4150027    46
4149989    46
4149925    46
3635864    46
3975295    46
Name: instructor_id, dtype: int64